In [1]:
import os
import time
# import utils
import tensorflow as tf

In [2]:
import numpy as np
import imageio as io


In [3]:
from load_data import load_batch,prepare_data
from model.mapnet import mapnet

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [5]:
import easydict

In [6]:
args = easydict.EasyDict({
        "batch_size": 4,
        "learning_rate": 0.001,
        "crop_height": 512,
        "crop_width": 512,
        "clip_size": 450,
        "num_epochs": 80,
        "h_flip": True,
        "v_flip": True,
        "color": True,
        "rotation": True,
        "start_valid": 20,
        "valid_step": 1
})

In [7]:
args

{'batch_size': 4,
 'learning_rate': 0.001,
 'crop_height': 512,
 'crop_width': 512,
 'clip_size': 450,
 'num_epochs': 80,
 'h_flip': True,
 'v_flip': True,
 'color': True,
 'rotation': True,
 'start_valid': 20,
 'valid_step': 1}

In [8]:
num_images=[]


In [9]:
train_img, train_label,valid_img,valid_lab= prepare_data()


In [10]:
num_batches=len(train_img)//(args.batch_size)

In [11]:
img=tf.placeholder(tf.float32,[None,args.crop_height,args.crop_width,3])
is_training=tf.placeholder(tf.bool)
label=tf.placeholder(tf.float32,[None,args.crop_height,args.crop_height,1])

In [12]:
pred=mapnet(img,is_training)
pred1=tf.nn.sigmoid(pred)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.Dense instead.



In [13]:
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

In [14]:
with tf.control_dependencies(update_ops):

    sig=tf.nn.sigmoid_cross_entropy_with_logits(labels=label, logits=pred)
    sigmoid_cross_entropy_loss = tf.reduce_mean(sig)
    train_step = tf.train.AdamOptimizer(args.learning_rate).minimize(sigmoid_cross_entropy_loss)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [15]:
saver=tf.train.Saver(var_list=tf.global_variables())

In [16]:
def load():
    import re
    print("Reading checkpoints...")
    checkpoint_dir = './checkpoint/'

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt and ckpt.model_checkpoint_path:
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        saver.restore(sess, os.path.join(checkpoint_dir, ckpt_name))
        counter = int(next(re.finditer("(\d+)(?!.*\d)",ckpt_name)).group(0))
        print("Checkpoint {} read Successed".format(ckpt_name))
        return True, counter
    else:
        print("Checkpoint not find ")
        return False, 0

In [17]:
def train():

    tf.global_variables_initializer().run()

    could_load, checkpoint_counter = load()
    if could_load:
        start_epoch = (int)(checkpoint_counter / num_batches)
        start_batch_id = checkpoint_counter - start_epoch * num_batches
        counter = checkpoint_counter
        print("Checkpoint Load Successed")

    else:
        start_epoch = 0
        start_batch_id = 0
        counter = 1
        print("train from scratch...")

    train_iter=[]
    train_loss=[]
    IOU=0.65
    # utils.count_params()
    print("Total train image:{}".format(len(train_img)))
    print("Total validate image:{}".format(len(valid_img)))
    print("Total epoch:{}".format(args.num_epochs))
    print("Batch size:{}".format(args.batch_size))
    print("Learning rate:{}".format(args.learning_rate))
    # print("Checkpoint step:{}".format(args.checkpoint_step))

    print("Data Argument:")
    print("h_flip: {}".format(args.h_flip))
    print("v_flip: {}".format(args.v_flip))
    print("rotate: {}".format(args.rotation))
    print("clip size: {}".format(args.clip_size))
    loss_tmp = []
    for i in range(start_epoch, args.num_epochs):
        epoch_time=time.time()
        id_list = np.random.permutation(len(train_img))

        for j in range(start_batch_id, num_batches):
            img_d = []
            lab_d = []

            for ind in range(args.batch_size):
                id = id_list[j * args.batch_size + ind]
                img_d.append(train_img[id])
                lab_d.append(train_label[id])

            x_batch, y_batch = load_batch(img_d, lab_d)
            feed_dict = {img: x_batch,
                         label: y_batch,
                         is_training:True
                         }

            _, loss, pred1 = sess.run([train_step, sigmoid_cross_entropy_loss, pred], feed_dict=feed_dict)

            loss_tmp.append(loss)
            if (counter % 100 == 0):
                tmp = np.median(loss_tmp)
                train_iter.append(counter)
                train_loss.append(tmp)
                print('Epoch', i, '|Iter', counter, '|Loss', tmp)
                loss_tmp.clear()

            counter += 1
        start_batch_id = 0
        print('Time:', time.time() - epoch_time)

        # saver.save(sess, './checkpoint/model.ckpt', global_step=counter)

        if (i>args.start_valid):
            if (i-args.start_valid)%args.valid_step==0:
                val_iou = validation()
                print("last iou valu:{}".format(IOU))
                print("new_iou value:{}".format(val_iou))
                if val_iou > IOU:
                    print("Save the checkpoint...")
                    saver.save(sess, './checkpoint/model.ckpt', global_step=counter, write_meta_graph=True)
                    IOU = val_iou
    saver.save(sess, './checkpoint/model.ckpt', global_step=counter)


In [18]:
def f_iou(predict, label):

    tp = np.sum(np.logical_and(predict == 1, label == 1))
    fp = np.sum(predict==1)
    fn = np.sum(label == 1)
    return tp,fp+fn-tp



In [19]:
def validation():

    print("validate...")
    inter=0
    unin=0
    for j in range(0,len(valid_img)):

        x_batch = valid_img[j]
        x_batch = io.imread(x_batch) / 255.0
        x_batch = np.expand_dims(x_batch, axis=0)
        feed_dict = {img: x_batch,
                     is_training:False

                     }

        predict = sess.run(pred1, feed_dict=feed_dict)

        predict[predict < 0.5] = 0
        predict[predict >= 0.5] = 1
        result = np.squeeze(predict)
        gt_value=io.imread(valid_lab[j])
        intr,unn=f_iou(gt_value,result)

        inter=inter+intr
        unin=unin+unn
    return inter*1.0/unin



In [20]:
with tf.Session() as sess:
    train()


Reading checkpoints...
Checkpoint not find 
train from scratch...
Total train image:6
Total validate image:5
Total epoch:80
Batch size:4
Learning rate:0.001
Data Argument:
h_flip: True
v_flip: True
rotate: True
clip size: 450


ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted: OOM when allocating tensor with shape[4,256,128,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node swap_in_gradients/batch_normalization_17/cond/FusedBatchNormV3_grad/FusedBatchNormGradV3_1}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Mean/_25]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted: OOM when allocating tensor with shape[4,256,128,128] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node swap_in_gradients/batch_normalization_17/cond/FusedBatchNormV3_grad/FusedBatchNormGradV3_1}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor